In [ ]:
import os
import cv2
import numpy as np
from retinaface import RetinaFace
from deepface import DeepFace
from scipy.spatial.distance import cosine

class FaceRecognitionSystem:
    def __init__(self, database_dir):
        """
        Initialize the face recognition system with a database of known faces
        
        Args:
            database_dir (str): Directory containing images of known individuals
        """
        self.database_dir = database_dir

        # Dictionaries to store embeddings and labels for different models
        self.embeddings = {
            'ArcFace': {'embeddings': [], 'labels': []},
            'Facenet': {'embeddings': [], 'labels': []},
            'VGGFace2': {'embeddings': [], 'labels': []}
        }

        # Load known faces
        self.load_database()

    def load_database(self):
        """
        Load face embeddings from images in the database directory
        """
        for filename in os.listdir(self.database_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Extract label from filename (assuming filename is the person's name)
                label = os.path.splitext(filename)[0]
                img_path = os.path.join(self.database_dir, filename)

                try:
                    # Read the image
                    img = cv2.imread(img_path)

                    # Detect faces
                    faces = RetinaFace.detect_faces(img)

                    if faces:
                        # Get the first detected face
                        face = faces[next(iter(faces))]
                        x1, y1, x2, y2 = face['facial_area']
                        face_img = img[y1:y2, x1:x2]

                        # Extract embeddings using different models
                        for model in ['ArcFace', 'Facenet', 'VGGFace2']:
                            embedding = self.extract_embedding(face_img, model)

                            if embedding is not None:
                                self.embeddings[model]['embeddings'].append(embedding)
                                self.embeddings[model]['labels'].append(label)

                        print(f"{label} has been registered")

                except Exception as e:
                    print(f"Error processing {filename}: {e}")

    def extract_embedding(self, face_img, model_name):
        """
        Extract face embedding using specified model
        
        Args:
            face_img (np.ndarray): Cropped face image
            model_name (str): Name of the face recognition model
        
        Returns:
            np.ndarray: Face embedding
        """
        try:
            embedding = DeepFace.represent(
                face_img,
                model_name=model_name,
                detector_backend="retinaface",
                enforce_detection=False,
                align=True
            )[0]['embedding']

            return np.array(embedding).astype('float32').flatten()

        except Exception as e:
            print(f"Error extracting {model_name} embedding: {e}")
            return None

    def predict_identity(self, query_face_img, threshold=0.5, weights=None):
        """
        Predict the identity of a query face using multiple models
        
        Args:
            query_face_img (np.ndarray): Cropped query face image
            threshold (float): Maximum distance for a match
            weights (dict): Weights for different models
        
        Returns:
            str: Predicted identity or 'Unknown'
        """
        # Default weights if not provided
        if weights is None:
            weights = {
                'ArcFace': 0.5,
                'Facenet': 0.3,
                'VGGFace2': 0.2
            }

        # Extract embeddings for the query face
        query_embeddings = {}
        for model in ['ArcFace', 'Facenet', 'VGGFace2']:
            embedding = self.extract_embedding(query_face_img, model)
            if embedding is not None:
                query_embeddings[model] = embedding

        # Aggregate predictions from different models
        predictions = {}
        for model, query_embedding in query_embeddings.items():
            # Calculate cosine distances
            distances = [cosine(query_embedding, emb) for emb in self.embeddings[model]['embeddings']]

            if distances:
                # Find the nearest neighbor
                nearest_index = np.argmin(distances)
                nearest_distance = distances[nearest_index]

                # Check if the distance is within the threshold
                if nearest_distance <= threshold:
                    label = self.embeddings[model]['labels'][nearest_index]
                    predictions[label] = predictions.get(label, 0) + weights.get(model, 0)

        # Return the most confident prediction
        if predictions:
            return max(predictions, key=predictions.get)

        return 'Unknown'

    def recognize_faces(self, image_path):
        """
        Recognize faces in a given image
        
        Args:
            image_path (str): Path to the input image
        
        Returns:
            list: List of recognized identities
        """
        # Read the image
        img = cv2.imread(image_path)

        # Detect faces
        faces = RetinaFace.detect_faces(img)

        recognized_faces = []

        if faces:
            for _, face in faces.items():
                x1, y1, x2, y2 = face['facial_area']
                face_img = img[y1:y2, x1:x2]

                # Predict identity
                identity = self.predict_identity(face_img)
                recognized_faces.append(identity)

                # Optional: Draw bounding box and identity on the image
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, identity, (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Optional: Display the image with recognized faces
            cv2.imshow('Face Recognition', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        return recognized_faces

# Example usage
if __name__ == "__main__":
    # Initialize the face recognition system with a database directory
    face_system = FaceRecognitionSystem("path/to/your/database/directory")

    # Recognize faces in a query image
    recognized_faces = face_system.recognize_faces("path/to/query/image.jpg")
    print("Recognized Faces:", recognized_faces)